Methodology was based on Section 6.2 of paper "Quantifying measurement quality and load distribution in Tor": https://dl.acm.org/doi/pdf/10.1145/3427228.3427238

* I can always create a new container image with a Tor process representing each client and it will always choose a new guard node, so I don't have to change the default Tor definitions
* Log all relays along with their data, specifically geographical location, used in each client circuit.
* In the paper, the authors logged over 8.6 million circuits, representing about 275,000 circuits per day
* Drop the circuit immediately after creation to ensure not overloading the guard relays
* Wait 16 minutes between each circuit

In [1]:
!pip install stem

In [2]:
import subprocess
import requests
import shlex
import traceback
import time
from stem import CircStatus
from stem.control import Controller
import stem.process
from flask import Flask
import os
import shutil


In [3]:
WEB_PORT = 8081

In [4]:
app = Flask(__name__)


@app.route('/')
def index():
  return "<h1>Hi Grandma!</h1>"

In [5]:

def connect_to_controller():
    with Controller.from_port() as controller:
        controller.authenticate()
        print("Successfully authenticated with Tor control port")

        hidden_service_dir = os.path.join(controller.get_conf('DataDirectory', '/tmp'), 'hello_world')

        # Create a hidden service where visitors of port 80 get redirected to local
        # port 8081 (this is where Flask runs by default).

        print(" * Creating our hidden service in %s" % hidden_service_dir)
        result = controller.create_hidden_service(hidden_service_dir, 80, target_port=WEB_PORT)

        # The hostname is only available when we can read the hidden service
        # directory. This requires us to be running with the same user as tor.

        if result.hostname:
            print(" * Our service is available at %s, press ctrl+c to quit" % result.hostname)
        else:
            print(" * Unable to determine our service's hostname, probably due to being unable to read the hidden service directory")

        try:
            app.run(port=WEB_PORT)
        finally:
            # Shut down the hidden service and clean it off disk. Note that you *don't*
            # want to delete the hidden service directory if you'd like to have this
            # same *.onion address in the future.
            print(" * Shutting down our hidden service")
            controller.remove_hidden_service(hidden_service_dir)
            shutil.rmtree(hidden_service_dir)
    

### Check if Tor is already running
First, restart the kernel. Then, run the following commands:
```
sudo lsof -i -P | grep LISTEN | grep 9050
sudo kill -9 95441
```

In [6]:
connect_to_controller()
    

Successfully authenticated with Tor control port
 * Creating our hidden service in /Users/danielalopes/Library/Application Support/TorBrowser-Data/Tor/hello_world


AttributeError: 'NoneType' object has no attribute 'hostname'